In [1]:
import random, time
from pprint import pprint

In [2]:
class VacuumCleanerAgent:
    def __init__(self, board_shape=(4,4), dirty_tile_ratio=0.4, starting_tile=None, starting_direction=None):
        
        starting_tile = starting_tile if starting_tile is not None else (random.randint(0, board_shape[0]-1), random.randint(0, board_shape[1]-1))
        starting_direction = starting_direction if starting_direction is not None else random.choice('news')
        
        self.arrows = {'n':'▲', 'e':'▶', 'w':'◀', 's':'▼'}
        self.dirty_arrows = {'n':'⇈', 'e':'⇉', 'w':'⇇', 's':'⇊'}
        self.next_direction = {'n':'e', 'e':'s', 's':'w', 'w':'n'}
        self.clear_tile = '□'
        self.traversed_tile = '◼'
        self.dirty_tile = 'x'
        
        self.init_board(board_shape, dirty_tile_ratio, starting_tile, starting_direction)
        self.cost = 0
        
    
    def init_board(self, board_shape, dirty_tile_ratio, starting_tile, starting_direction):
        self.tile = starting_tile
        self.direction = starting_direction
        self.arrow = self.arrows[self.direction]
        
        self.board_shape = board_shape
        self.board = [[None for _ in range(board_shape[1])] for _ in range(board_shape[0])]
        self.board[self.tile[0]][self.tile[1]] = self.arrow
        self.dirty_tiles_num = 0
        self.sucked_tiles_num = 0
        
        for i in range(board_shape[0]):
            for j in range(board_shape[1]):
                if self.board[i][j] == self.arrow:
                    continue
                self.board[i][j] = self.dirty_tile if random.random()<= dirty_tile_ratio else self.clear_tile
                if self.board[i][j] == self.dirty_tile:
                    self.dirty_tiles_num += 1
                
        
    def draw_board(self):
        pprint(self.board)
        print(self.tile)
        print()
        time.sleep(0.5)
        
        
    def board_size(self):
        return self.board_shape[0] * self.board_shape[1]
        
        
    def next_tile(self, curr_tile, direction):
        match direction:
            case 'n':
                next_tile = (curr_tile[0]-1, curr_tile[1])
            case 'e':
                next_tile = (curr_tile[0], curr_tile[1]+1)
            case 'w':
                next_tile = (curr_tile[0], curr_tile[1]-1)
            case 's':
                next_tile = (curr_tile[0]+1, curr_tile[1])
        return next_tile
        
        
    def move(self):
        self.board[self.tile[0]][self.tile[1]] = self.dirty_tile if self.arrow == self.dirty_arrows[self.direction] else self.traversed_tile
        
        self.tile = self.next_tile(self.tile, self.direction)
        
        curr_tile = self.board[self.tile[0]][self.tile[1]]
        self.arrow = self.dirty_arrows[self.direction] if curr_tile == self.dirty_tile else self.arrows[self.direction]
        self.board[self.tile[0]][self.tile[1]] = self.arrow
        
        self.cost += 1
        
        
    def turn_right(self):
        self.direction = self.next_direction[self.direction]
        self.arrow = self.arrows[self.direction] if self.arrow in self.arrows.values() else self.dirty_arrows[self.direction]
        self.board[self.tile[0]][self.tile[1]] = self.arrow
        
        
    def dirt_detected(self):
        return self.arrow in self.dirty_arrows.values()
    
    
    def suck(self):
        self.arrow = self.arrows[self.direction]
        self.board[self.tile[0]][self.tile[1]] = self.arrow
        self.cost += 2
        self.sucked_tiles_num += 1
    

In [3]:
# turning to the direction with less tiles
def solve_tttdwlt(agent: VacuumCleanerAgent):
    traversed_tiles = []
    while len(traversed_tiles) < agent.board_size():
        curr_tile = agent.tile
        traversed_tiles.append(curr_tile)
        agent.draw_board()
        
        if agent.dirt_detected():
            agent.suck()
            agent.draw_board()
            
        candidate_directions = {
            'n': curr_tile[0],
            'e': agent.board_shape[1] - 1 - curr_tile[1],
            'w': curr_tile[1],
            's': agent.board_shape[0] - 1 - curr_tile[0]
            }
        candidate_directions = {k:v for k,v in candidate_directions.items() if v != 0}
        selected_direction = None
        while len(candidate_directions)>0:
            selected_direction = min(candidate_directions, key=candidate_directions.get)
            next_tile = agent.next_tile(curr_tile, selected_direction)
            if next_tile not in traversed_tiles:
                break
            del candidate_directions[selected_direction]
        
        if selected_direction == None:
            break
        
        while agent.direction != selected_direction:
            agent.turn_right()
            agent.draw_board()
            
        agent.move()
    
    pprint({'Dirty Tiles Generated': agent.dirty_tiles_num, 'Sucked Tiles':agent.sucked_tiles_num, 'Total Cost':agent.cost})
        
        
        

In [ ]:
agent = VacuumCleanerAgent()
solve_tttdwlt(agent)